# Problem Statement:

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

* Thumbs up:  Increase the volume
* Thumbs down: Decrease the volume
* Left swipe: 'Jump' backwards 10 seconds
* Right swipe: 'Jump' forward 10 seconds  
* Stop: Pause the movie

# Data Understanding:

* The training data consists of a few hundred videos categorised into one of the five classes.
* Each video (typically 2-3 seconds long) is divided into a sequence of 30 frames(images).
*  videos have two types of dimensions - either 360x360 or 120x160 (depending on the webcam used to record the videos)


* Data can be downloaded from: https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL

# 1. Importing Neccessary libraries

In [1]:
# Importing the necessary libraries
import numpy as np
import os
#from scipy.misc.pilutil import imread, imresize
import datetime
import os
import warnings
warnings.filterwarnings("ignore")
import abc
from sys import getsizeof

In [2]:
from imageio import imread
from skimage.transform import resize
#from scipy.misc import imread, imresize
from PIL import Image, ImageFilter, ImageEnhance

In [3]:
!pip install tensorflow==2.09

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [5]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# 2. Importing data

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
import gdown

In [7]:
file_id = "1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL"
file_name = "Project_data.zip"
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_name)

Downloading...
From (original): https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
From (redirected): https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL&confirm=t&uuid=f16ae1fb-9711-4606-b6e1-182931694057
To: /content/Project_data.zip
100%|██████████| 1.71G/1.71G [00:44<00:00, 38.1MB/s]


'Project_data.zip'

In [8]:
# Remove extracted files if already exist
!rm -rf 'Project_data'
# Unzip file
!unzip -qq Project_data.zip

In [9]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

In [10]:
train_doc.shape

(663,)

In [11]:
val_doc.shape

(100,)

# 3. Generator Function

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`, `z` and normalization such that you get high accuracy.

In [12]:
img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29] #create a list of image numbers you want to use for a particular video
frames = len(img_idx)
shape_h = 120
shape_w = 120

In [ ]:
# # generator with augmentation for train data
# def generator(source_path, folder_list, batch_size, is_train = False, augmention = False, debug=False):
#     # print('\nSource path = ', source_path, '; batch size =', batch_size)
#     x = len(img_idx)
#     y, z = shape_h, shape_w
#     while True:
#         # doubling the data for augmentation
#         if is_train and augmention:
#             t = np.concatenate((np.random.permutation(folder_list), np.random.permutation(folder_list)))
#         else:
#             t = np.random.permutation(folder_list)

#         if (len(t)%batch_size) == 0:
#             num_batches = int(len(t)/batch_size)
#         else:
#             num_batches = len(t)//batch_size + 1

#         for batch in range(num_batches): # we iterate over the number of batches
#             batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
#             batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
#             for folder in range(batch_size): # iterate over the batch_size
#                 if debug:
#                     plt.figure(figsize=(20,5))
#                 #handling remaining datapoints
#                 folder_idx = folder + (batch*batch_size)
#                 if folder_idx >= len(t):
#                     break
#                 folder_str = t[folder_idx]
#                 imgs = os.listdir(source_path+'/'+ folder_str.split(';')[0]) # read all the images in the folder
#                 # randomly enabling augmentation and augmentation type
#                 aug_type = None
#                 if is_train and augmention and rn.randint(0,1) == 1:
#                     aug_type = rn.randint(0, 4) #randomly selecting augmentation type
#                 for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
#                     image = imread(source_path+'/'+ folder_str.strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

#                     # plotting original images for debugging purpose only
#                     if debug:
#                         plt.subplot(2, x, idx+1)
#                         plt.imshow(image.astype('uint8'))

#                     #crop the images and resize them. Note that the images are of 2 different shape
#                     #and the conv3D will throw error if the inputs in a batch have different shapes
#                     # making the rectangle images into square by cropping sides
#                     # so the aspect ration can be mantained while resizing.
#                     if image.shape[1] > image.shape[0]:
#                         diff_px = image.shape[1] - image.shape[0]
#                         crop_start = diff_px//2
#                         crop_end = crop_start + image.shape[0]
#                         image = image[:, crop_start:crop_end]
#                     elif image.shape[0] > image.shape[1]:
#                         diff_px = image.shape[0] - image.shape[1]
#                         crop_start = diff_px//2
#                         crop_end = crop_start + image.shape[1]
#                         image = image[:, crop_start:crop_end]

#                     resized_im = resize(image, (y,z))

#                     if aug_type is not None:
#                         if aug_type == 0: # edge Enhancement
#                             resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.EDGE_ENHANCE))
#                         elif aug_type == 1: # adding gaussian blur
#                             resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.GaussianBlur(1)))
#                         elif aug_type == 2: # enchancing image detailing
#                             resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.DETAIL))
#                         elif aug_type == 3: # sharpening image
#                             resized_im = np.array(Image.fromarray(resized_im, 'RGB').filter(ImageFilter.SHARPEN))
#                         elif aug_type == 4: # Brightness enhancement
#                             resized_im = np.array(ImageEnhance.Brightness((Image.fromarray(resized_im, 'RGB'))).enhance(1.5))
#                     # plotting rezised images for debugging purpose only
#                     if debug:
#                         plt.subplot(2, x, idx+x+1)
#                         plt.imshow(resized_im)

#                     batch_data[folder,idx,:,:,0] = resized_im[:,:,0]/255 #normalise and feed in the image
#                     batch_data[folder,idx,:,:,1] = resized_im[:,:,1]/255 #normalise and feed in the image
#                     batch_data[folder,idx,:,:,2] = resized_im[:,:,2]/255 #normalise and feed in the image

#                 batch_labels[folder, int(folder_str.strip().split(';')[2])] = 1
#             yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


In [13]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        # left over batches which should be handled separately
        leftover_batches = len(t) - num_batches * batch_size

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size, len(img_idx), shape_h, shape_w, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image, (shape_h, shape_w))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        # write the code for the remaining data points which are left after full batches
        if leftover_batches != 0:
            for batch in range(num_batches):
                # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_data = np.zeros((batch_size,len(img_idx),shape_h, shape_w,3))
                # batch_labels is the one hot representation of the output: 10 videos with 5 columns as classes
                batch_labels = np.zeros((batch_size,5))
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path +'/'+t[batch * batch_size + folder].split(';')[0])
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                        image = imread(source_path +'/'+t[batch * batch_size + folder].split(';')[0] +'/'+imgs[item]).astype(np.float32)
                        image = resize(image, (shape_h,shape_w))

                        batch_data[folder,idx,:,:,0] = (image[:,:,0])/255
                        batch_data[folder,idx,:,:,1] = (image[:,:,1])/255
                        batch_data[folder,idx,:,:,2] = (image[:,:,2])/255

                    #Fill the one hot encoding stuff where we maintain the label
                    batch_labels[folder, int(t[batch * batch_size + folder].split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [15]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


# 4. Model

Here you make the model using different functionalities that Keras provides. Remember to use Conv3D and MaxPooling3D and not Conv2D and Maxpooling2D for a 3D convolution model. You would want to use TimeDistributed while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [16]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from tensorflow.keras.layers import Conv2D,Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

## 3D Convolutional Neural Networks (Conv3D) Model

In [31]:
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(len(img_idx),120,120,3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [32]:
sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())
# #optimiser = 'adam' #write your optimizer
# #model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# #print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 18, 120, 120, 64)  5248      
                                                                 
 batch_normalization_4 (Batc  (None, 18, 120, 120, 64)  256      
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 18, 120, 120, 64)  0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 9, 60, 120, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 9, 60, 120, 128)   221312    
                                                                 
 batch_normalization_5 (Batc  (None, 9, 60, 120, 128) 

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [33]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [34]:
model_name = 'model_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}..keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
#

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [36]:
num_epochs=25

In [37]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                     callbacks=callbacks_list, validation_data=val_generator,
                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/25
21/21 [==============================] - ETA: 0s - loss: 4.6548 - categorical_accuracy: 0.3661 Source path =  /content/Project_data/val ; batch size = 32

Epoch 1: saving model to model_Conv3D_2024-08-0916_28_11.225274/model-00001-4.65485-0.36607-1.89518-0.25781..keras
21/21 [==============================] - 243s 12s/step - loss: 4.6548 - categorical_accuracy: 0.3661 - val_loss: 1.8952 - val_categorical_accuracy: 0.2578 - lr: 0.0010
Epoch 2/25
21/21 [==============================] - ETA: 0s - loss: 1.3439 - categorical_accuracy: 0.5536 
Epoch 2: saving model to model_Conv3D_2024-08-0916_28_11.225274/model-00002-1.34391-0.55357-3.45089-0.30469..keras
21/21 [==============================] - 242s 12s/step - loss: 1.3439 - categorical_accuracy: 0.5536 - val_loss: 3.4509 - val_categorical_accuracy: 0.3047 - lr: 0.0010
Epoch 3/25
21/21 [==============================] - ETA: 0s - loss: 1.2514 - categorical_accuracy: 0

## CNN2D + ConvLSTM2D

In [17]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=(len(img_idx),120,120,3))
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
)
model.add(BatchNormalization())
model.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Dense(64, activation='relu'))
)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [18]:
optimiser = optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())
#sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 18, 118, 118, 8)  224       
 ibuted)                                                         
                                                                 
 batch_normalization (BatchN  (None, 18, 118, 118, 8)  32        
 ormalization)                                                   
                                                                 
 time_distributed_1 (TimeDis  (None, 18, 116, 116, 16)  1168     
 tributed)                                                       
                                                                 
 batch_normalization_1 (Batc  (None, 18, 116, 116, 16)  64       
 hNormalization)                                                 
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 114, 114, 8)       6

In [ ]:
# train_generator = generator(train_path, train_doc, batch_size, is_train = True, augmention = enable_augmentation)
# val_generator = generator(val_path, val_doc, batch_size)

In [19]:
batch_size=32

In [20]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
model_name = 'model_init_CNN2D+convLSTM' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'CNN2D+convLSTM-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}..keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
#

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.001) #0.00001
callbacks_list = [checkpoint, LR]

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
num_epochs=50

In [24]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.3301 - categorical_accuracy: 0.3720Source path =  /content/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_CNN2D+convLSTM_2024-08-1002_21_21.466451/CNN2D+convLSTM-00001-1.33005-0.37202-1.80222-0.21875..keras
21/21 [==============================] - 140s 7s/step - loss: 1.3301 - categorical_accuracy: 0.3720 - val_loss: 1.8022 - val_categorical_accuracy: 0.2188 - lr: 0.0100
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.1155 - categorical_accuracy: 0.4940
Epoch 2: saving model to model_init_CNN2D+convLSTM_2024-08-1002_21_21.466451/CNN2D+convLSTM-00002-1.11550-0.49405-2.32022-0.20312..keras
21/21 [==============================] - 137s 7s/step - loss: 1.1155 - categorical_accuracy: 0.4940 - val_loss: 2.3202 - val_categorical_accuracy: 0.2031 - lr: 0.0100
Epoch 3/50
21/21 [==============================] - ETA: 0s 

# Conclusions:

After iterations with trail and error, two models are considered Conv3D and combination of Conv2D and ConvLSTM. various Iterations of batch_size , Epochs, learning rate are considered to get the best outcome

It is observed that, Conv2D + ConvLSTM gives the best results with maximum training accuracy of 94% and validation accuracy of 84%.

Hence, Conv2D + ConvLSTM model with batch_size of 32, number of epochs as 50, learning rate of 0.01 with Adam as optimizer is best choice for the problem statement